In [368]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Embedding, concatenate, Flatten, Dropout, BatchNormalization
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.callbacks import EarlyStopping

In [401]:
data = pd.read_csv('/Users/student/code/Liedjieboer/forward_farming/clean_data/winemag-data-130k-v2.csv')

In [402]:
data = data[['country', 'points', 'description']]

In [407]:
data[data['country'] == 'England']

,country,points,description
4073,England,90,A mix of fruity notes immediately hit: white c...
6700,England,91,Crisp green apple flavors are immediately appa...
9909,England,95,"Touches of vanilla, cream and lovely hints of ..."
22156,England,91,Golden color and hints of oatmeal on the nose ...
22369,England,94,"Pure notes of red apple rise from the glass, p..."
...,...,...,...
124898,England,93,Creaminess and stone are all that is signalled...
124922,England,89,The nose tells of the crisp green freshness of...
124924,England,89,There is a flinty touch of reduction on the no...
128098,England,89,Creamy notes of tart red currant and crisp red...


In [195]:
def encode(val):
    if val < 86:
        return 'bad'
    
    elif 86 <= val <= 92:
        return 'good'
    
    else:
        return 'excellent'

In [408]:
data['points'] = data['points'].apply(encode)

In [144]:
data.shape

(129971, 3)

In [221]:
data['points'].value_counts()

good         95348
bad          21960
excellent    12663
Name: points, dtype: int64

In [199]:
Ohe = OneHotEncoder(sparse=False)

In [201]:
country_OHE = Ohe.fit_transform(data[["country", "points"]])

In [205]:
lst_sent = data["description"].to_list()

In [207]:
scaled_sentence = [text_to_word_sequence(x)for x in lst_sent]

In [212]:
word2vec = Word2Vec(sentences=scaled_sentence, vector_size=60, min_count=10, window=10)

In [213]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, scaled_sentence)

In [308]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=100)
X_train_pad.shape

(129971, 100, 60)

In [301]:
X_train_pad[0].shape

(65, 60)

In [363]:
num_tags = 47  # Number of unique issue tags  # Size of vocabulary obtained when preprocessing text data

title_input = keras.Input(
    shape=(10,60), name="title"
)  # Variable-length sequence of ints
  # Variable-length sequence of ints
tags_input = keras.Input(
    shape=(num_tags,), name="tags"
)  # Binary vectors of size `num_tags`


# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = layers.LSTM(128)(title_input)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features,  tags_input])

# Stick a department classifier on top of the features
department_pred = layers.Dense(60*90, name="department")(x)
final_output_layer = layers.Reshape((90,60), input_shape= (60*90,))(department_pred)

# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(
    inputs=[title_input, tags_input],
    outputs=[final_output_layer],
)

In [364]:
model.compile(optimizer="adam", loss = "mse")

In [361]:
X_train_true = np.array([X_train_pad[i][:10] for i in range(0,129971)])

In [311]:
X_train_true.shape

(129971, 3, 60)

In [362]:
y = np.array([X_train_pad[i][10:] for i in range(0,129971)])

In [312]:
y.shape

(129971, 97, 60)

In [373]:
es = EarlyStopping(patience = 3, restore_best_weights = True, monitor = 'loss')

In [444]:
model.fit(x = [X_train_true, country_OHE], y = y, epochs=100, callbacks = [es])

Epoch 1/100
4062/4062 [==============================] - 187s 46ms/step - loss: 1.6163
Epoch 2/100
4062/4062 [==============================] - 166s 41ms/step - loss: 1.6161
Epoch 3/100
4062/4062 [==============================] - 151s 37ms/step - loss: 1.6159
Epoch 4/100
4062/4062 [==============================] - 162s 40ms/step - loss: 1.6156
Epoch 5/100
4062/4062 [==============================] - 154s 38ms/step - loss: 1.6154
Epoch 6/100
4062/4062 [==============================] - 150s 37ms/step - loss: 1.6153
Epoch 7/100
4062/4062 [==============================] - 151s 37ms/step - loss: 1.6151
Epoch 8/100
4062/4062 [==============================] - 159s 39ms/step - loss: 1.6149
Epoch 9/100
4062/4062 [==============================] - 144s 36ms/step - loss: 1.6148
Epoch 10/100
4062/4062 [==============================] - 146s 36ms/step - loss: 1.6146
Epoch 11/100
4062/4062 [==============================] - 156s 38ms/step - loss: 1.6144
Epoch 12/100
4062/4062 [=================

In [298]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 100, 60)]    0           []                               
                                                                                                  
 lstm_12 (LSTM)                 (None, 128)          96768       ['title[0][0]']                  
                                                                                                  
 tags (InputLayer)              [(None, 47)]         0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 175)          0           ['lstm_12[0][0]',                
                                                                  'tags[0][0]']             

In [299]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [436]:
lst = []
lst.append(word2vec.wv['the'])
lst.append(word2vec.wv['wine'])
lst.append(word2vec.wv['has'])
lst.append(word2vec.wv['a'])
lst.append(word2vec.wv['full'])
lst.append(word2vec.wv['body'])
lst.append(word2vec.wv['with'])
lst.append(word2vec.wv['notes'])
lst.append(word2vec.wv['of'])
lst.append(word2vec.wv['cream'])

sentence_dummy = np.array(lst)


In [437]:
sentence_dummy.reshape(1,10,60)

array([[[-1.5492995e+00,  1.9984275e+00, -2.2047281e+00, -9.8416859e-01,
          4.0636425e+00,  2.7126929e-01, -3.0776113e-01, -8.0594409e-01,
          8.3566123e-01, -1.5392100e+00, -3.4448320e-01,  1.5546471e-01,
         -2.1625874e+00, -3.2390130e-01,  2.4263144e+00,  2.6277950e-01,
          3.8651752e-01, -1.8618387e-01,  2.7404194e+00,  1.5126208e+00,
         -1.4293277e+00,  1.0313597e+00,  1.0607793e+00, -6.7597145e-01,
         -1.4032891e+00,  4.1763788e-01,  2.3677871e+00, -8.8272578e-01,
          9.8124897e-01,  2.5763211e+00,  1.2614928e+00,  3.7713704e+00,
         -2.8052120e+00, -2.4427965e-01,  1.5355296e+00,  7.2721517e-01,
         -2.3449163e+00, -1.8851796e+00, -3.9920516e-02,  1.3156286e+00,
          2.2418332e+00,  2.9242113e+00, -1.5167044e-01, -8.1673449e-01,
         -1.6051136e+00, -8.9713335e-01,  4.0738778e+00, -2.2531400e+00,
         -5.3778875e-01, -9.8846805e-01,  1.9412811e+00,  4.6243012e-01,
         -2.1752014e+00, -9.9211931e-01,  6.5951347

In [431]:
data[(data['country'] == 'England') & (data['points'] == 'bad')]

,country,points,description


In [427]:
country_OHE[4073].reshape(1,47)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [433]:
bad_arr = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [440]:
exe_arr = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [446]:
good_arr = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [451]:
pred = model.predict([sentence_dummy.reshape(1,10,60),exe_arr.reshape(1,47)])

1/1 [==============================] - 0s 36ms/step


In [411]:
pred.shape

(1, 90, 60)

In [453]:
full_string = []

for x in range(90):
    word = word2vec.wv.most_similar(pred[0,x,:],topn=1)
    full_string.append(word)

final_string = []
    
for i in [x[0][0] for x in full_string]:
    if i != 'and':
        final_string.append(i)

final_string

['quince',
 'quince',
 'quince',
 'quince',
 'the',
 'rounded',
 'the',
 'the',
 'ripe',
 'the',
 'the',
 'that',
 'it',
 'the',
 'the',
 'the',
 'crisp',
 'the',
 'the',
 'the',
 'lively',
 'freshness',
 'fresh',
 'crisp',
 'freshness',
 'freshness',
 'its',
 'still',
 'it',
 'ample',
 'that',
 'structure',
 'balanced',
 'age',
 'leaving',
 'more',
 'age',
 'hold',
 'lemon',
 'on',
 'finish',
 'finish',
 'you',
 'with',
 'its',
 'drink',
 'invigorating',
 'nature',
 'drink',
 '2017–2027',
 'neighbor',
 'at',
 'least',
 "syrah's",
 'washington',
 'commercial',
 'drink',
 'cortona',
 'you']